In [1]:
'''CODE HERE'''
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
# import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
# from sklearn.linear_model import LogisticRegression
# #A random forest is a meta estimator that fits a number of decision tree classifiers 
# #on various sub-samples of the dataset and use averaging to improve the predictive 
# #accuracy and control over-fitting.
# from sklearn.ensemble import RandomForestClassifier
# #a discriminative classifier formally defined by a separating hyperplane.
# from sklearn.svm import SVC
# #displayd data
# from IPython.display import display
# %matplotlib inline

df = pd.read_csv('Data/finalData.csv')
def cleanDates():
    for index,row in df.iterrows():
        date = row['Date'].split("/")
        date[2] ="20" +date[2]
        date = date[2] + "-" + date[1] + "-" + date[0]
        row['Date'] = date
        df.loc[index] = row
        
cleanDates()

In [2]:

##returns the matches between start date and end date// yyyy-mm-dd
##display(getMatchesBeforGivenDate("2008-08-16","2010-08-17"))
def getMatchesBeforGivenDate(data,start_date,end_date):
    mask = (data['Date'] <= end_date)
    return data[mask]

df['Date'] = pd.to_datetime(df['Date'],infer_datetime_format=True)


In [3]:
startDate="2005-01-01"
def getMatchesWithHomeTeam(data, teamName):
    return data.loc[data['HomeTeam'] == teamName]


def getMatchesWithAwayTeam(data, teamName):
    return data.loc[data['AwayTeam'] == teamName]


def getMatchesWithHomeTeamWin(data):
    return data.loc[data['FTR'] == "H"]


def getMatchesWithAwayTeamWin(data):
    return data.loc[data['FTR'] == "A"]


def getMatchesWithDraw(data):
    return data.loc[data['FTR'] == "D"]


def getHomeTeamWinPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    homeTeamWinRows = getMatchesWithHomeTeamWin(teamRows)
    return len(homeTeamWinRows) / len(teamRows)


def getHomeTeamDrawPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    homeTeamDrawRows = getMatchesWithDraw(teamRows)
    return len(homeTeamDrawRows) / len(teamRows)


def getAwayTeamWinPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    awayTeamWinRows = getMatchesWithAwayTeamWin(teamRows)
    return len(awayTeamWinRows) / len(teamRows)


def getAwayTeamDrawPercent(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    awayTeamDrawRows = getMatchesWithDraw(teamRows)
    return len(awayTeamDrawRows) / len(teamRows)


def getAvgHomeTeamGoals(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalHomeGoals = teamRows['FTHG'].sum()
    # display(teamRows)
    return totalHomeGoals / len(teamRows)


def getAvgAwayTeamGoals(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalAwayGoals = teamRows['FTAG'].sum()
    # display(teamRows)
    return totalAwayGoals / len(teamRows)


# getAvgHomeTeamGoals("Arsenal","2008-09-30")

def getAvgHomeTeamShots(teamName, toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalhomeshots = teamRows['HS'].sum()
    display(teamRows)
    return totalhomeshots / len(teamRows)


def getAverageHomeFeature(teamName, toDate, feature):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalHomeFeature = teamRows[feature].sum()
    # display(teamRows)
    return totalHomeFeature / len(teamRows)


def getAverageAwayFeature(teamName, toDate, feature):
    matchesInDateRange = getMatchesBeforGivenDate(df, startDate, toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange, teamName)
    if len(teamRows) is 0:
        return 0;
    totalAwayFeature = teamRows[feature].sum()
    # display(teamRows)
    return totalAwayFeature / len(teamRows)



In [4]:
##Driver code
feature_vector ={}
for index,row in df.iterrows():
    home_team=row['HomeTeam']
    away_team =row['AwayTeam']
    endTime= row['Date']
    
    ##Home team stats
    
    ##1. Home win %
    h_home_win = getHomeTeamWinPercent(home_team,endTime)
    h_home_draw = getHomeTeamDrawPercent(home_team,endTime)

    ##2  Away win %
    h_away_win = getAwayTeamWinPercent(home_team,endTime)
    h_away_draw = getAwayTeamDrawPercent(home_team,endTime)
    
    ##3. Average home goals
    h_home_goals = getAvgHomeTeamGoals(home_team, endTime)
    ##4. Average away goals
    h_away_goals = getAvgAwayTeamGoals(home_team, endTime)
    
    ##5. Last few games - form 
    
    ##6. HS = Home Team Shots - take average
    h_home_shots = getAverageHomeFeature(home_team, endTime, "HS") 
    ##7. AS = Away Team Shots - take average
    h_away_shots = getAverageAwayFeature(home_team, endTime, "AS") 

    ##8. HST = Home Team Shots on Target -- take average
    h_home_shots_target =getAverageHomeFeature(home_team, endTime, "HST")
    ##9. AST = Away Team Shots on Target - - take average
    h_away_shots_target =getAverageHomeFeature(home_team, endTime, "AST")
    
#     ##10. HHW = Home Team Hit Woodwork- take average
#     h_home_shots_wood =getAverageHomeFeature(home_team, endTime, "HHW")
#     ##11. AHW = Away Team Hit Woodwork- take average
#     h_away_shots_wood =getAverageHomeFeature(home_team, endTime, "AHW")
    
    ##12. HC = Home Team Corners- take average
    h_home_corners =getAverageHomeFeature(home_team, endTime, "HC")    
    ##13. AC = Away Team Corners- take average
    h_away_corners =getAverageHomeFeature(home_team, endTime, "AC")

    ##14. HF = Home Team Fouls Committed- take average
    h_home_fouls =getAverageHomeFeature(home_team, endTime, "HF")
    ##15. AF = Away Team Fouls Committed- take average
    h_away_fouls =getAverageAwayFeature(home_team, endTime, "AF")

#     ##16. HFKC = Home Team Free Kicks Conceded- take average
#     h_home_fk =getAverageHomeFeature(home_team, endTime, "HFKC")
#     ##17. AFKC = Away Team Free Kicks Conceded- take average
#     h_away_fk =getAverageHomeFeature(home_team, endTime, "AFKC")

#     ##18. HO = Home Team Offsides- take average
#     h_home_off =getAverageHomeFeature(home_team, endTime, "HO")
#     ##19. AO = Away Team Offsides- take average
#     h_away_off =getAverageAwayFeature(home_team, endTime, "AO")
    
    ##20. HY = Home Team Yellow Cards- take average
    h_home_yellow, h_away_yellow = getAverageHomeFeature(home_team, endTime, "HY"),getAverageAwayFeature(home_team,endTime,"AY")
    ##21. AY = Away Team Yellow Cards- take average
    ##22. HR = Home Team Red Cards- take average
    h_home_red, h_away_red = getAverageHomeFeature(home_team, endTime, "HR"),getAverageAwayFeature(home_team,endTime,"AR")
    ##23. AR = Away Team Red Cards- take average
    total_yellow= h_home_yellow + h_away_yellow
    total_Red = h_home_red + h_away_red 
    ##24. HBP = Home Team Bookings Points (10 = yellow, 25 = red)- take average
    h_home_foul_points = total_yellow * 10
    ##25. ABP = Away Team Bookings Points (10 = yellow, 25 = red)- take average
    h_away_foul_points = total_Red *25
    feature_vector[index] =[h_home_win, h_home_draw, h_away_win, h_away_draw,h_home_goals, h_away_goals,
                            h_home_shots, h_away_shots, h_home_shots_target, h_away_shots_target,
                             h_home_corners, h_away_corners, h_home_fouls,
                            h_away_fouls,  h_home_foul_points, 
                            h_away_foul_points]
    
    print(feature_vector[index])
    
    

[1.0, 0.0, 0, 0, 1.0, 0, 24.0, 0, 14.0, 4.0, 7.0, 5.0, 11.0, 0, 0.0, 0.0]
[1.0, 0.0, 0, 0, 3.0, 0, 14.0, 0, 8.0, 2.0, 4.0, 3.0, 13.0, 0, 10.0, 0.0]
[0.0, 0.0, 0, 0, 2.0, 0, 10.0, 0, 5.0, 11.0, 3.0, 5.0, 11.0, 0, 20.0, 0.0]
[1.0, 0.0, 0, 0, 2.0, 0, 11.0, 0, 6.0, 6.0, 5.0, 6.0, 10.0, 0, 30.0, 0.0]
[1.0, 0.0, 0, 0, 2.0, 0, 14.0, 0, 10.0, 5.0, 7.0, 9.0, 11.0, 0, 10.0, 0.0]
